# "Building a self driving car - Software Configuration"
> "Preparng software while waiting for the parts to arrive. Lets begin..."
- toc: true
- branch: master
- badges: false
- comments: true
- categories: [self driving, software]
- hide: false
- search_exclude: false
- metadata_key1: MUSHR
- metadata_key2: Software

# Purpose

The intention of this page is NOT to replace the existing documentation by MUSHR. Please follow them [here](https://mushr.io), buit visit this page if things dont go as planned. 

![](https://abhisheksreesaila.github.io/blog/images/sw_process_flowchart.png "Process Flow")


# Scenarios

## Monitor not booting up, after MUSHR image is installed. How do i proceed?

**Issue**

When you install the stock image of JETSON NANO using [BelenaEtcher](https://www.balena.io/etcher), connect JETSON NANO using HDMI, NAno boots and you can see the Linux GUI. But, if you do the same exact same thing using [MUSHR IMAGE](https://drive.google.com/file/d/17Xn63ZWA_e3DVxt_ajhzu-uKW0hUG4iX/view?usp=sharing), it does not show the GUI. without the GUI, software config is almost impossible (*well, I am not referring to LINUX gurus here, everything possible from a GUI is also possible with command line in LINUX*). In other words, we need the GUI working.


**Solution**

> Important: If you have a monitor that supports DISPLAY PORT, use it!! Instant solution. (Time saving : 3-4 days)

If you only have HDMI port, read on.



To troubleshoot the issue, 

You need to LOGIN (using SSH). For SSH, you need WIFI, For WIFI you need to LOGIN. Get it! chicken-egg problem? Almost. There is one possible solution, **Hard-wired USB connection**

- Connect the **MICRO USB** port of NANO to the **USB PORT** of your laptop (also known as HEADLESS LOGIN)

**HEADLESS LOGIN (on a Mac)**

```bash

 #Go to dev directory
cd /dev  

 #check for all things like tty and usb
ls *tty*usb*       
tty.usbmodem146123  # result (yours might be slightly different)

 #use the screen command with the result of previous step + serial interface number
screen tty.usbmodem146123 115200  #  115200 is the Jetson serial interface. "screen" is installed on a mac by default

# (It will ask for SUDO login and password)

```

**ACTIVATE THE ROOT in JETSON NANO**

Be aware that the root account is not active by default. You will need to do:

```bash
sudo passwd root
```

Then go ahead to set password. Use the root user using

```bash
su root
```

**USE WPA_SUPPLICANT TO CONNECT** 

- My references
 - [Wiki](https://wiki.archlinux.org/index.php/Wpa_supplicant)
 - [bbs.archlinux.org](https://bbs.archlinux.org/viewtopic.php?id=234877)
 - [linuxcommando](https://linuxcommando.blogspot.com/2013/10/how-to-connect-to-wpawpa2-wifi-network.html)

```bash

# CREATE a file at */etc/wpa_supplicant/wpa_supplicant.conf*

vi /etc/wpa_supplicant/wpa_supplicant.conf

# PRESS "I" to start insert mode  

# Add this content to the file

ctrl_interface=/run/wpa_supplicant
update_config=1

# PRESS ":wq" to save and quit the editor

# Then, run this command

wpa_supplicant -B -i wlan0 -c /etc/wpa_supplicant/wpa_supplicant.conf

wpa_cli -i wlan0

# This will present an interactive prompt (>), which has tab completion and descriptions of completed commands.

scan

# Ok
# <3>CTRL-EVENT-SCAN-RESULTS

scan_results

# bssid / frequency / signal level / flags / ssid
# 00:00:00:00:00:00 2462 -49 [WPA2-PSK-CCMP][ESS] MYSSID
# 11:11:11:11:11:11 2437 -64 [WPA2-PSK-CCMP][ESS] ANOTHERSSID

add_network
# 0

set_network 0 ssid "MYSSID"

set_network 0 psk "passphrase"

enable_network 0

#<2>CTRL-EVENT-CONNECTED - Connection to 00:00:00:00:00:00 completed (reauth) [id=0 id_str=]

save_config
# OK

quit
 
```


**Check if the wireless device is up**

```bash
ip link show wlan0    #Look for the word "UP" inside the brackets in the first line of the output.

/sbin/iw wlan0 link  # Connected to <your network>>

ping google.com    # testng if it connects

ifconfig  #GET the IP. Needed to connect to SSH. Uff, finally!

```

## Connect to the JETSON NANO

Open a terminal window

```bash
ssh robot@IP_ADDRESS  #enter your IP address

```
Open another terminal window
```bash
ssh robot@IP_ADDRESS  #enter your IP address
dmesg −−follow
````

Plug the HDMI cable to JETSON NANO

## HDID ERROR


# Testing 

## Testing Bluetooth

```bash

jstest-gtk

```
    

 

```bash
export ROS_IP=<<YOUR_IP_ADDRESS>>
export ROS_MASTER_URI=http://<<YOUR_IP_ADDRESS>>:11311
```
    